# Representing Salient Context

This notebook creates a smoothed average for each of the 64 input measures for every postcode, comprising values from those postcodes found within each 160m buffer. 

The `darribas/gds_py:6.1` Docker container was used to create a computational environment.

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [2]:
geom = gpd.read_parquet('postcode_geom_osgb.pq')

/opt/conda/lib/python3.8/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [3]:
%time buffered = geom.buffer(160, cap_style=3)
%time buff_idx, pt_idx = geom.geometry.sindex.query_bulk(buffered, predicate="contains")

CPU times: user 8.66 s, sys: 78.2 ms, total: 8.73 s
Wall time: 8.73 s
CPU times: user 4.47 s, sys: 240 ms, total: 4.71 s
Wall time: 4.71 s


In [32]:
df = pd.read_csv('sentinel_gb_encoder_map_3_20210317.csv')

In [33]:
%time latent = np.take(df.values, pt_idx, axis=0)

CPU times: user 1 s, sys: 896 ms, total: 1.9 s
Wall time: 1.9 s


In [34]:
%time group = pd.DataFrame(latent).groupby(buff_idx)

CPU times: user 291 ms, sys: 24 ms, total: 315 ms
Wall time: 314 ms


In [35]:
%time smoothed = group.mean()

CPU times: user 3.53 s, sys: 397 ms, total: 3.93 s
Wall time: 3.93 s


In [37]:
smoothed.to_csv("sentinel_gb_encoder_map_3_20210317_smoothed.csv")